In [1]:
import Algos
#from Algos import ANN as t
#from Algos import PolynomialFeatures as t
#from Algos import RandomForest as t
#from Algos import RNN as t
#from Algos import CausalDiscovery as t
from Algos import TCN as t


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from keras.models import clone_model
from keras import backend as K 

import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from matplotlib import pyplot as plt

2025-05-19 20:18:22.623164: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 20:18:22.628894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-19 20:18:22.636572: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-19 20:18:22.638712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 20:18:22.645031: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Read the time series data for a specific building (Building_1) from a CSV file using pandas' read_csv function.
# The file likely contains information such as electricity consumption, production, or related metrics for Building 1.
# Save the filepath of building 2 and 3 for later use
building1_filepath="citylearn_challenge_2023_phase_1/Building_1.csv"
building2_filepath="citylearn_challenge_2023_phase_1/Building_2.csv"
building3_filepath="citylearn_challenge_2023_phase_1/Building_3.csv"


electricity_data = pd.read_csv(building1_filepath)
carbon_data = pd.read_csv("citylearn_challenge_2023_phase_1/carbon_intensity.csv")
pricing_data = pd.read_csv("citylearn_challenge_2023_phase_1/pricing.csv")
weather_data = pd.read_csv("citylearn_challenge_2023_phase_1/weather.csv")

dataframe = pd.concat([electricity_data, carbon_data, pricing_data, weather_data], axis=1)


#print(dataframe)

target = 'cooling_demand'
#target = 'solar_generation'

features = ['month', 'hour', 'day_type', 'daylight_savings_status',
            'indoor_dry_bulb_temperature', 'average_unmet_cooling_setpoint_difference', 
            'indoor_relative_humidity', 'non_shiftable_load', 'dhw_demand', 
            'heating_demand', 'solar_generation', 'occupant_count', 
            'indoor_dry_bulb_temperature_set_point', 'hvac_mode', 
            'carbon_intensity', 'electricity_pricing', 
            'electricity_pricing_predicted_6h', 'electricity_pricing_predicted_12h', 
            'electricity_pricing_predicted_24h', 'outdoor_dry_bulb_temperature', 
            'outdoor_relative_humidity', 'diffuse_solar_irradiance', 
            'direct_solar_irradiance', 'outdoor_dry_bulb_temperature_predicted_6h', 
            'outdoor_dry_bulb_temperature_predicted_12h', 
            'outdoor_dry_bulb_temperature_predicted_24h', 
            'outdoor_relative_humidity_predicted_6h', 
            'outdoor_relative_humidity_predicted_12h', 
            'outdoor_relative_humidity_predicted_24h', 
            'diffuse_solar_irradiance_predicted_6h', 
            'diffuse_solar_irradiance_predicted_12h', 
            'diffuse_solar_irradiance_predicted_24h', 
            'direct_solar_irradiance_predicted_6h', 
            'direct_solar_irradiance_predicted_12h', 
            'direct_solar_irradiance_predicted_24h']


In [3]:
# This should make the model/training deterministic, but it does not work
#seed = 7
#keras.utils.set_random_seed(seed)
    
#"parameters" for how the model is build
number_of_entries=-1
filters=512
kernel_size=3
WINDOW_SIZE=168
FORECAST_HORIZON=24
dense_size=100
#end "parameters"

number_of_models=10

dataframe_target = dataframe[target].copy()
#use crete_sliding_window
X, y = t.create_sliding_windows(dataframe_target.values, WINDOW_SIZE, FORECAST_HORIZON,number_of_entries)

# Split into train and test sets
X_train, X_test, y_train, y_test = t.split_train_test(X, y, 0.2)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Further split the training set into training and validation sets.
# 80% of the training data is used for training, and 20% is used for validation.
# This helps in tuning hyperparameters and assessing model performance during training.
X_train, X_val, y_train, y_val = t.split_train_test(X_train, y_train, 0.2)

_model = t.create_TCN_model(dense_size, filters, kernel_size,WINDOW_SIZE)

best_model,history = t.Train_TCN_model(X_train,y_train,X_val,y_val,_model)

best_real = y_test
best_pred = t.Predict_TCN_model(best_model,X_test)

best_STD = np.std(best_real-best_pred)
best_NMAE = mean_absolute_error(best_real,best_pred)/np.mean(best_real)
best_NSQRT = mean_squared_error(best_real,best_pred)/np.mean(best_real)

print("Model n.",0," STD = ",best_STD," NMAE = ",best_NMAE, " NSQRT = ",best_NSQRT)

K.clear_session()


#for number_of_models times create and train models and find the best

for index in range(number_of_models-1):
    model = clone_model(_model)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_absolute_error"])
    dataframe_target = dataframe[target].copy()
    #use crete_sliding_window
    X, y = t.create_sliding_windows(dataframe_target.values, WINDOW_SIZE, FORECAST_HORIZON,number_of_entries)

    # Split into train and test sets
    X_train, X_test, y_train, y_test = t.split_train_test(X, y, 0.2)

    # Scaling the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    # Further split the training set into training and validation sets.
    # 80% of the training data is used for training, and 20% is used for validation.
    # This helps in tuning hyperparameters and assessing model performance during training.
    X_train, X_val, y_train, y_val = t.split_train_test(X_train, y_train, 0.2)

    model,history = t.Train_TCN_model(X_train,y_train,X_val,y_val,model)

    real = y_test
    pred = t.Predict_TCN_model(model,X_test)

    better_number = 0
    STD = np.std(real-pred)
    if(STD<best_STD):
        better_number+=1
    NMAE = mean_absolute_error(real,pred)/np.mean(real)
    if(NMAE<best_NMAE):
        better_number+=1
    NSQRT = mean_squared_error(real,pred)/np.mean(real)
    if(NSQRT<best_NSQRT):
        better_number+=1
    if(better_number>=2):
        best_model = model
        best_STD = STD
        best_NMAE = NMAE
        best_NSQRT = NSQRT
        best_real =real
        best_pred = pred
    print("Model n.",index+1," STD = ",STD," NMAE = ",NMAE, " NSQRT = ",NSQRT)
    K.clear_session()

/home/enrico/anaconda3/envs/Casual/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747678704.027695  134396 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1747678704.046317  134396 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1747678704.048980  

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step
Model n. 0  STD =  0.9694495300285912  NMAE =  0.19556954422899706  NSQRT =  0.27841637877398634
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step 
Model n. 1  STD =  1.0640014334108483  NMAE =  0.21588096872191334  NSQRT =  0.33010065344021317
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step 
Model n. 2  STD =  0.9129633288536491  NMAE =  0.19218334057741568  NSQRT =  0.24753543999341296
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step 
Model n. 3  STD =  1.90780331773912  NMAE =  0.9107990471816134  NSQRT =  4.167532418894234
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step 
Model n. 4  STD =  0.9825381885731995  NMAE =  0.1882765338092134  NSQRT =  0.2661643011988538
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step 
Model n. 5  STD =  0.9062333273650645  NMAE =  0.2005320518403123  NSQRT =  0.2434873367074987
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step 
Model n. 6  STD =  1.7408530368335668  NMAE =  0.9905526101262035  NSQRT =  4.089250338488442


2025-05-19 20:20:35.679737: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 32.81MiB (rounded to 34406400)requested by op 
2025-05-19 20:20:35.679808: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2025-05-19 20:20:35.679819: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 604, Chunks in use: 604. 151.0KiB allocated for chunks. 151.0KiB in use in bin. 2.6KiB client-requested in use in bin.
2025-05-19 20:20:35.679826: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 47, Chunks in use: 47. 23.8KiB allocated for chunks. 23.8KiB in use in bin. 18.5KiB client-requested in use in bin.
2025-05-19 20:20:35.679831: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (1024): 	Total Chunks: 2, Chunks in use: 2. 2.8KiB allocated for chunks. 2.8KiB in use in bin. 2.3KiB client-requested in use in bin.
2025-05-

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/enrico/anaconda3/envs/Casual/lib/python3.10/asyncio/base_events

In [ ]:
# Plot a scatter plot of the true test data against the predicted values.
# - The scatter plot shows the relationship between the actual values and the predicted values.
# - The title of the plot displays the Mean Absolute Error (MAE), the Standard Deviation (STD) and the Normalized Mean Absolute Error.
print("filter = ",filters)
print("kernel_size = ",kernel_size)
print("window size = ",WINDOW_SIZE)
print("forecast horizon = ",FORECAST_HORIZON)
print("dense size = ",dense_size)
plt.scatter(best_real, best_pred)
plt.title(r"MAE = %.2f" % (mean_absolute_error(best_real,best_pred)) + r" STD = %.2f" % best_STD+ r" NMAE = %.2f" % best_NMAE + r" NSQRT = %.2f"%best_NSQRT)

# Plot a reference line where predicted values equal true values (y = x).
# This line helps to visualize how well the predicted values match the true values.
plt.plot(best_real, best_real, 'k-')

# Label the x-axis as 'True test data' and the y-axis as 'Predicted test data'.
plt.xlabel('True test data')
plt.ylabel('Predicted test data')

# Display the plot to the user.
plt.show()



In [ ]:
#Do the prediction for building 2 using the model and training orbtained from building 1. 
electricity_data2 = pd.read_csv(building2_filepath)
dataframe2 = pd.concat([electricity_data, carbon_data, pricing_data, weather_data], axis=1)
X, y = t.create_sliding_windows(dataframe[target].values, WINDOW_SIZE, FORECAST_HORIZON,number_of_entries)

# Split into train and test sets
X_train, X_test, y_train, y_test = t.split_train_test(X, y, 0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
                                
pred = t.Predict_TCN_model(best_model,X_test)
real=y_test

STD = np.std(real-pred)
NMAE = mean_absolute_error(real,pred)/np.mean(real)
NSQRT = mean_squared_error(real,pred)/np.mean(real)
                                

In [ ]:
#Same plot as the one from building 1 (maybe create a function to not repeat code)
# Plot a scatter plot of the true test data against the predicted values.
# - The scatter plot shows the relationship between the actual values and the predicted values.
# - The title of the plot displays the Mean Absolute Error (MAE), the Standard Deviation (STD) and the Normalized Mean Absolute Error.
print("filter = ",filters)
print("kernel_size = ",kernel_size)
print("window size = ",WINDOW_SIZE)
print("forecast horizon = ",FORECAST_HORIZON)
print("dense size = ",dense_size)
plt.scatter(real, pred)
plt.title(r"MAE = %.2f" % (mean_absolute_error(best_real,best_pred)) + r" STD = %.2f" % STD+ r" NMAE = %.2f" % NMAE + r" NSQRT = %.2f"%NSQRT)
# Plot a reference line where predicted values equal true values (y = x).
# This line helps to visualize how well the predicted values match the true values.
plt.plot(real, real, 'k-')

# Label the x-axis as 'True test data' and the y-axis as 'Predicted test data'.
plt.xlabel('True test data')
plt.ylabel('Predicted test data')

# Display the plot to the user.
plt.show()